<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Clima/Clima_Espa%C3%B1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PARA IPHONE VERSIÓN 2 ESPAÑA

import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# === 1. Cargar shapefile ===
ruta_shapefile = "/content/drive/MyDrive/Clima_Jalisco/Clima_España.shp"
gdf = gpd.read_file(ruta_shapefile)

# === 2. Verificar las columnas disponibles ===
print("Columnas disponibles en el shapefile:")
for col in gdf.columns:
    print(f"  - {col}")

# === 3. Identificar la columna de orden numérico ===
columnas_posibles = ['Orden', 'orden', 'Codigo', 'codigo', 'Clave', 'clave', 'Code', 'code']
columna_orden = None

for col in columnas_posibles:
    if col in gdf.columns:
        columna_orden = col
        break

if columna_orden is None:
    for col in gdf.columns:
        sample_values = gdf[col].dropna().head(5)
        if all(str(val).replace('.', '').isdigit() for val in sample_values if val != ''):
            columna_orden = col
            break

if columna_orden is None:
    print("No se encontró una columna de orden numérico. Usando orden alfabético.")
    categorias_ordenadas = sorted(gdf['simple'].unique())
else:
    print(f"Usando columna de orden: '{columna_orden}'")

    orden_map = gdf[[columna_orden, 'simple', 'Unidos']].drop_duplicates()

    try:
        orden_map['orden_numeric'] = pd.to_numeric(orden_map[columna_orden])
        orden_map = orden_map.sort_values('orden_numeric')
    except:
        orden_map = orden_map.sort_values(columna_orden)

    categorias_ordenadas = orden_map['simple'].tolist()

    print(f"\nOrden detectado ({len(categorias_ordenadas)} categorías):")
    for i, cat in enumerate(categorias_ordenadas, 1):
        print(f"  {i}. {cat}")

# === 4. Generar paleta de colores ===
n_categorias = len(categorias_ordenadas)
cmap = plt.cm.Spectral_r
colores = [mcolors.rgb2hex(cmap(i / max(1, n_categorias - 1))) for i in range(n_categorias)]

# Crear diccionario de colores
categorias_color = {cat: colores[i] for i, cat in enumerate(categorias_ordenadas)}

# === 5. Crear mapa CON MEJORES CONFIGURACIONES PARA SAFARI ===
bounds = gdf.total_bounds
centro = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# Configuración específica para Safari/iPhone
mapa = folium.Map(
    location=centro,
    zoom_start=5,
    tiles="OpenStreetMap",
    # Configuraciones optimizadas para iOS
    zoomControl=True,
    scrollWheelZoom=True,
    dragging=True,
    touchZoom=True,
    doubleClickZoom=False,
    boxZoom=False,
    keyboard=False,
    tap=False,
    prefer_canvas=True,  # Mejor rendimiento en iOS
    control_scale=True
)

# === 6. Estilo usando la columna 'simple' para colorear ===
def style_function(feature):
    categoria = feature["properties"]["simple"]
    color = categorias_color.get(categoria, "#999999")
    return {
        "fillColor": color,
        "color": "grey",
        "weight": 0.2,
        "fillOpacity": 0.70
    }

# Tooltip optimizado para iOS
tooltip = folium.GeoJsonTooltip(
    fields=["Unidos"],
    aliases=["Detallada:"],
    style=("""
        background-color: white;
        color: #333333;
        font-family: -apple-system, BlinkMacSystemFont, sans-serif;
        font-size: 12px;
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 8px;
        max-width: 250px;
        white-space: normal;
        word-wrap: break-word;
        -webkit-user-select: none;
        user-select: none;
    """),
    sticky=True
)

folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=tooltip
).add_to(mapa)

# === 7. Leyenda CON SCROLL FUNCIONAL PARA SAFARI/IPHONE ===
items_leyenda = ""
for i, cat in enumerate(categorias_ordenadas, 1):
    color = categorias_color[cat]
    items_leyenda += f"""<div style='margin-bottom:3px;line-height:16px;'>
        <i style='background:{color};width:16px;height:14px;display:inline-block;
        margin-right:5px;border:1px solid #666;vertical-align:middle;border-radius:2px;'></i>
        <span style='font-size:10px;'><b>{i}.</b> {cat}</span>
    </div>"""

# Leyenda con scroll completamente funcional para Safari
legend_html = f"""
<div id="safari-legend" style="
    position: absolute;
    top: 90px;
    left: 10px;
    width: 280px;
    max-width: 75vw;
    z-index: 9999;
    background: rgba(255,255,255,0.98);
    border: 2px solid #333;
    border-radius: 12px;
    padding: 10px;
    box-shadow: 0 4px 20px rgba(0,0,0,0.3);
    font-family: -apple-system, BlinkMacSystemFont, sans-serif;
">
    <!-- Cabecera con botón de toggle -->
    <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 8px;">
        <div style="font-size:15px; font-weight:bold;">Climas de España Peninsular y Baleares</div>
        <button id="legend-toggle" style="
            background: #007AFF;
            color: white;
            border: none;
            border-radius: 12px;
            padding: 4px 10px;
            font-size: 10px;
            cursor: pointer;
            -webkit-appearance: none;
        ">▼</button>
    </div>

    <div style="font-size:12px; color:#666; margin-bottom:8px; border-bottom:1px solid #ddd; padding-bottom:8px;">
        Clasificación Climática (Thornthwaite, 1948)
    </div>

    <!-- Contenido principal CON SCROLL MEJORADO -->
    <div id="legend-main-content" style="
        max-height: 50vh;
        overflow-y: scroll;
        -webkit-overflow-scrolling: touch;
        padding-right: 5px;
    ">
        {items_leyenda}

        <!-- Pie de página -->
        <div style="
            margin-top: 12px;
            padding-top: 10px;
            border-top: 1px solid #999;
            font-size: 10px;
            color: #666;
            line-height: 1.2;
        ">
            Cálculo y edición cartográfica:<br>
            Armando Chávez Hernández<br>

            <br>
            Fuente: Capas raster de temperatura y precipitación<br>
            Karger, D.N., Conrad, O., Böhner, J., Kawohl, T., Kreft, H., Soria-Auza, R.W.,
            Zimmermann, N.E., Linder, H.P. & Kessler, M. (2021) Climatologies at high resolution
            for the earth’s land surface areas. EnviDat. https://doi.org/ 10.16904/envidat.228.v2.1
        </div>
    </div>
</div>

<style>
    /* Reset específico para Safari */
    #safari-legend {{
        -webkit-user-select: none;
        user-select: none;
        -webkit-tap-highlight-color: transparent;
        -webkit-touch-callout: none;
    }}

    /* Scroll optimizado para iOS */
    #legend-main-content {{
        scroll-behavior: smooth;
        overflow-y: scroll !important;
        -webkit-overflow-scrolling: touch !important;
    }}

    /* Scrollbar para WebKit (Safari, Chrome) */
    #legend-main-content::-webkit-scrollbar {{
        width: 6px;
    }}

    #legend-main-content::-webkit-scrollbar-track {{
        background: #f1f1f1;
        border-radius: 3px;
    }}

    #legend-main-content::-webkit-scrollbar-thumb {{
        background: #888;
        border-radius: 3px;
    }}

    #legend-main-content::-webkit-scrollbar-thumb:hover {{
        background: #555;
    }}

    /* Scrollbar para Firefox */
    #legend-main-content {{
        scrollbar-width: thin;
        scrollbar-color: #888 #f1f1f1;
    }}

    .legend-collapsed #legend-main-content {{
        display: none;
    }}

    /* Estado colapsado */
    .legend-collapsed {{
        height: auto !important;
        max-height: 80px !important;
    }}
</style>

<script>
// Script MEJORADO para scroll en Safari/iPhone
document.addEventListener('DOMContentLoaded', function() {{
    const legend = document.getElementById('safari-legend');
    const content = document.getElementById('legend-main-content');
    const toggleBtn = document.getElementById('legend-toggle');
    let isCollapsed = false;
    let isScrolling = false;

    // Función para toggle
    function toggleLegend() {{
        isCollapsed = !isCollapsed;
        if (isCollapsed) {{
            legend.classList.add('legend-collapsed');
            toggleBtn.textContent = '►';
        }} else {{
            legend.classList.remove('legend-collapsed');
            toggleBtn.textContent = '▼';
        }}
    }}

    // Evento para el botón
    toggleBtn.addEventListener('click', function(e) {{
        e.preventDefault();
        e.stopPropagation();
        toggleLegend();
    }});

    // MEJORA CRÍTICA: Manejo simplificado de eventos táctiles
    legend.addEventListener('touchstart', function(e) {{
        // Solo detener propagación si no es en el área de scroll
        if (!e.target.closest('#legend-main-content')) {{
            e.stopPropagation();
        }}
    }}, {{ passive: true }});

    legend.addEventListener('touchmove', function(e) {{
        // Permitir que el scroll funcione naturalmente
        if (e.target.closest('#legend-main-content')) {{
            // Estamos en el área de scroll, permitir el comportamiento normal
            return;
        }}
        e.stopPropagation();
    }}, {{ passive: true }});

    legend.addEventListener('touchend', function(e) {{
        if (!e.target.closest('#legend-main-content')) {{
            e.stopPropagation();
        }}
    }}, {{ passive: true }});

    // Forzar el estilo de scroll táctil en iOS
    function enableTouchScroll() {{
        if (content) {{
            content.style.overflowY = 'scroll';
            content.style.webkitOverflowScrolling = 'touch';

            // Asegurar que el contenido sea scrollable
            if (content.scrollHeight <= content.clientHeight) {{
                content.style.overflowY = 'hidden';
            }} else {{
                content.style.overflowY = 'scroll';
            }}
        }}
    }}

    // Inicializar scroll táctil
    enableTouchScroll();

    // Revisar periódicamente si el contenido necesita scroll
    setInterval(enableTouchScroll, 1000);

    // Ajustar dinámicamente la altura máxima
    function adjustMaxHeight() {{
        const viewportHeight = window.innerHeight;
        content.style.maxHeight = Math.min(viewportHeight * 0.6, 400) + 'px';
    }}

    // Ajustar al cargar y al cambiar tamaño
    window.addEventListener('load', adjustMaxHeight);
    window.addEventListener('resize', adjustMaxHeight);
    adjustMaxHeight();
}});

// Inicialización segura para Safari
function initLegend() {{
    // Asegurar que el mapa esté visible
    const mapContainer = document.querySelector('.folium-map');
    if (mapContainer) {{
        mapContainer.style.visibility = 'visible';
        mapContainer.style.opacity = '1';
    }}

    // Forzar redibujado para Safari
    const legend = document.getElementById('safari-legend');
    if (legend) {{
        legend.style.display = 'none';
        setTimeout(() => {{
            legend.style.display = 'block';
        }}, 10);
    }}
}}

if (document.readyState === 'loading') {{
    document.addEventListener('DOMContentLoaded', initLegend);
}} else {{
    initLegend();
}}
</script>
"""

mapa.get_root().html.add_child(folium.Element(legend_html))

# === 8. METADATOS MEJORADOS PARA SAFARI ===
mapa.get_root().header.add_child(folium.Element(
    '''
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no, viewport-fit=cover">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black-translucent">
    <meta name="format-detection" content="telephone=no">
    <style>
        /* Reset completo para Safari */
        html, body {
            width: 100% !important;
            height: 100% !important;
            margin: 0 !important;
            padding: 0 !important;
            overflow: hidden !important;
            -webkit-text-size-adjust: 100% !important;
            touch-action: pan-x pan-y !important;
            position: fixed !important;
            top: 0 !important;
            left: 0 !important;
            right: 0 !important;
            bottom: 0 !important;
        }

        /* Contenedor del mapa */
        .folium-map {
            width: 100% !important;
            height: 100% !important;
            position: fixed !important;
            top: 0 !important;
            left: 0 !important;
            z-index: 1 !important;
            visibility: visible !important;
            opacity: 1 !important;
        }

        /* Asegurar que la leyenda esté por encima */
        #safari-legend {
            z-index: 9999 !important;
            position: absolute !important;
        }

        /* Prevenir problemas de renderizado en Safari */
        * {
            -webkit-tap-highlight-color: transparent;
            -webkit-touch-callout: none;
        }

        /* Scroll global mejorado para iOS */
        .scrollable {
            -webkit-overflow-scrolling: touch !important;
            overflow-scrolling: touch !important;
        }

        /* Ajustes para notch de iPhone */
        @supports (padding: max(0px)) {
            .folium-map {
                padding-top: env(safe-area-inset-top);
                padding-bottom: env(safe-area-inset-bottom);
                padding-left: env(safe-area-inset-left);
                padding-right: env(safe-area-inset-right);
            }

            #safari-legend {
                top: max(90px, env(safe-area-inset-top));
                left: max(10px, env(safe-area-inset-left));
            }
        }
    </style>
    '''
))

# === 9. Guardar ===
ruta_salida = "/content/drive/MyDrive/Clima_Jalisco/Clima_España_Mapa_Safari.html"
mapa.save(ruta_salida)
print(f"\n✓ Mapa optimizado para Safari/iPhone guardado: {ruta_salida}")

print("\nResumen:")
print(f"- Categorías en la leyenda: {len(categorias_ordenadas)}")

print("\n✅ MEJORAS PARA SCROLL EN IPHONE:")
print("   👆 Scroll táctil completamente funcional")
print("   📱 Manejo mejorado de eventos táctiles")
print("   🔧 Script optimizado específicamente para Safari")
print("   📐 Altura responsive que se adapta al dispositivo")
print("   🎯 Detección inteligente de áreas scrollables")

mapa